 # MTH8408 : Méthodes d'optimisation et contrôle optimal

 ## Projet de session - ABDILLAHI & BÉLANGER

# 1re partie
Modélisation du problème
\begin{alignat}{2}
    \min_{(x_0, y_0, z_0), P_T, Q_i} \quad & C(t_k) = \sum_{i=1}^{N} C_{0,i}(t_k)\\
    \textrm{s.t.} \quad & 0 \leq P_T \leq P_T^{MAX}\\
                        & C(t_k) \leq C^{MAX}\\
                        & 0 < T_i(t_k) \leq C_{0,i}(t_k) , \quad && i \in \{1,\ldots,N\}\\
                        & Q_i(t_k) \geq 0 , \quad && i \in \{1,\ldots,N\} \\
                        & D_i(t_k) < H , \quad && i \in \{1,\ldots,N\} \\
                        & 0 \leq x_i \leq x^{MAX} , \quad && i \in \{1,\ldots,N\} \\
                        & 0 \leq y_i \leq y^{MAX} , \quad && i \in \{1,\ldots,N\} \\
                        & 0 \leq z_i \leq z^{MAX} , \quad && i \in \{1,\ldots,N\} \\
                        & (0,i), (i,0) \in E(t_k) , \quad && i \in \{1,\ldots,N\} \\
                        & (x_0-x_i)^2 + (y_0-y_i)^2 + (z_0-z_i)^2 > M^2 , \quad && i \in \{1,\ldots,N\}
\end{alignat}
avec ADNLP.

In [77]:
# using Pkg; 
# # Pkg.activate("jump")
# Pkg.add("JuMP")
# using JuMP
# Pkg.add("Ipopt")
# using Ipopt
# using MathOptInterface
# # Pkg.activate("nlpmodels")
# Pkg.add("ADNLPModels")
# using ADNLPModels
# Pkg.add("NLPModels")
# using NLPModels
# Pkg.add("NLPModelsJuMP")
# using NLPModelsJuMP

In [78]:
# f = 5250*10^6
# B = 160*10^6
# P_N = -85
# c = 3*10^8

# x0 = 46.4
# y0 = 12.3
# z0 = 10


const x_arr = [50 75 25]
const y_arr = [75 25 25] 
const z_arr = [10 10 10]

1×3 Matrix{Int64}:
 10  10  10

La capacité se calcule à l'aide du théorème de Shannon-Hartley:
$$
    C_{0,i} = B\log_2(1+SNR_i)
$$
Le rapport signal sur bruit s'obtient quant à lui à l'aide du modèle de propagation de Friis:
$$
    \text{SNR}_i = P_T - 20\log_{10}(d_i) - 20\log_{10}(f) - 20\log_{10}(\frac{4\pi}{c}) - P_N
$$
où $f$ et $P_N$ sont donnés, $c$ est la vitesse de la lumière dans le vide et 
$$
    d_i = \sqrt{(x_i-x_0)^2 + (y_i-y_0)^2 + (z_i-z_0)^2}.
$$

In [79]:
using Pkg, ADNLPModels, NLPModelsIpopt, ForwardDiff
# Pkg.update()
Pkg.add("ADNLPModels")
Pkg.add("NLPModelsIpopt")

Pkg.add("Ipopt")
Pkg.add("JuMP")


   Resolving package versions...


  No Changes to `C:\Users\olivi\MTH8408\Project.toml`
  No Changes to `C:\Users\olivi\MTH8408\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\olivi\MTH8408\Project.toml`
  No Changes to `C:\Users\olivi\MTH8408\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\olivi\MTH8408\Project.toml`
  No Changes to `C:\Users\olivi\MTH8408\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\olivi\MTH8408\Project.toml`
  No Changes to `C:\Users\olivi\MTH8408\Manifest.toml`

In [80]:
function distance(i, x0::T, y0::T, z0::T) where T
    return sqrt((x_arr[i] - x0)^2 + (y_arr[i] - y0)^2 + (z_arr[i] - z0)^2)
end

distance (generic function with 5 methods)

In [81]:
function SNR(P_T::T, x0::T, y0::T, z0::T, i::Int) where T
    return P_T .- 20 * log10(distance(i, x0, y0, z0)) .- 20 * log10(f) .- 20 * log10(4 * pi / c) .- P_N
end

SNR (generic function with 6 methods)

In [ ]:
# Constants

const x_arr = [50 75 25]
const y_arr = [75 25 25] 
const z_arr = [10 10 10]

N = 3
P_max = 100
C_max = 10^9
x_max = 75
y_max = 75
z_max = 20
H = 10
M = 5
f = 5250*10^6
B = 160*10^6
P_N = -85
c = 3*10^8

In [ ]:
# Initial guess
x0_arr = [0.0, 0.0, 0.0, 0.0, 45.0, 12.0, 10.0]

#Lower and upper bounds
#P_T, Q[1:3], x0, y0, z0
lvar = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
uvar = [P_max, Inf, Inf, Inf, x_max, y_max, z_max]

#Constraints 
lcon = [0.0]
ucon = [C_max]

In [82]:
# Objective function
function objective_function(x)
    P_T = x[1]
    Q = x[2:4]
    x0, y0, z0 = x[5], x[6], x[7]
    cap_sum = 0.0
    for i in 1:N
        cap_sum += B * log2(1 + 10^(SNR(P_T, x0, y0, z0, i) / 10))
    end
    return cap_sum
end

# Constraints function
function constr(G, x)
    P_T = x[1]
    Q = x[2:4]
    x0, y0, z0 = x[5], x[6], x[7]
    cap_sum = 0.0
    for i in 1:N
        G += B * log2(1 + 10^(SNR(P_T, x0, y0, z0, i) / 10))
    end
end

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       28

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2609803e+09 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
# Create ADNLPModel
model = ADNLPModel(objective_function, x0_arr, lvar, uvar, con=constr, lcon=lcon, ucon=ucon)

# Define solver and solve the model
output = ipopt(model)

# Print results
println("Objective value: ", objective_function(output.solution))
println("P_T: ", output.solution[1])
println("Q0: ", output.solution[2])
println("Q1: ", output.solution[3])
println("Q2: ", output.solution[4])
println("x0: ", output.solution[5])
println("y0: ", output.solution[6])
println("z0: ", output.solution[7])